### N Gram Analysis According To Fourgram

In [35]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [36]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [37]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [38]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [39]:
def strip(df,column_list):
    '''
    strip(df,column_list) df is dataframe and column_list are selected columns \n
    strip(df_twogram_used,df_twogram_used.columns)
    '''
    for i in column_list:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [40]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [41]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [42]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Ngram Read

In [43]:
df_twogram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.xlsx")
df_twogram_used = strip(df_twogram_used,df_twogram_used.columns)
df_twogram_used.drop_duplicates(inplace=True)
df_twogram_used.reset_index(drop=True, inplace=True)
df_twogram_used

,twogram
0,bir şey
1,bu kadar
2,ne kadar
3,ve de
4,ne için
...,...
152,senin gibi
153,sorun değil
154,sorun yok
155,tamam mı


In [44]:
df_threegram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Used.xlsx")
df_threegram_used = strip(df_threegram_used, df_threegram_used.columns)
df_threegram_used.drop_duplicates(inplace=True)
df_threegram_used.reset_index(drop=True, inplace=True)
df_threegram_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
333,bir daha hiç
334,bir de şu
335,bir daha asla
336,peki ne için


In [45]:
twogram_used_list = list(set(df_twogram_used["twogram"]))
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [46]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
220163,dışarıda bizimle,6
220164,bizim kimseye,6
220165,tarafa bunu,6
220166,beri kahrolası,6


In [47]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
500343,sonra insanlar bu,10
500344,o anda dünya,10
500345,zaman zaman sizinle,10
500346,bir güç sahibi,10


In [48]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


In [49]:
twogram_list = df_twogram["twogram"].to_list()
threegram_list = df_threegram["threegram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

#### Give Reveal Course

##### Twogram In Threegram In Selected Fourgram (1B)

In [50]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [51]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_in_selected_four

,threegram,fourgram
0,ne kadar güzel,ne kadar güzel bir
1,biliyorsun değil mi,bunu biliyorsun değil mi
2,bu kadar çok,neden bu kadar çok
3,ben de seni,ben de seni seviyorum
4,şey var mı,bir şey var mı
5,evet ben de,evet ben de öyle
6,bir şey var,bir şey var mı
7,bir şey var,gereken bir şey var
8,bir şey var,başka bir şey var
9,bir şey var,istediğim bir şey var


In [52]:
df_threegram_in_selected_four["fourgram"].nunique()

43

In [53]:
df_threegram_in_selected_four2 = df_threegram_in_selected_four[["threegram"]]
df_threegram_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_in_selected_four2.reset_index(drop=True, inplace=True)
df_threegram_in_selected_four2

<ipython-input-53-b6cd2c5833f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threegram_in_selected_four2.drop_duplicates(inplace=True)


,threegram
0,ne kadar güzel
1,biliyorsun değil mi
2,bu kadar çok
3,ben de seni
4,şey var mı
5,evet ben de
6,bir şey var
7,güzel değil mi
8,sen de biliyorsun
9,bu da ne


In [54]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [55]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_in_threegram_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_in_threegram_in_selected_four2

,twogram,threegram
0,ben de,ben de seni
1,ben de,evet ben de
2,ben de,ben de öyle
3,ne kadar,ne kadar güzel
4,çok güzel,çok güzel değil
5,uzun zaman,çok uzun zaman
6,benim için,benim için çok
7,uzun zaman,uzun zaman oldu
8,bu kadar,bu kadar çok
9,teşekkür ederim,çok teşekkür ederim


In [56]:
df_twogram_in_three_in_selected_four = pd.merge(df_twogram_in_threegram_in_selected_four2, df_threegram_in_selected_four, how="inner", on="threegram")
df_twogram_in_three_in_selected_four.drop_duplicates(inplace=True)
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,ben de,ben de seni,ben de seni seviyorum
1,ben de,evet ben de,evet ben de öyle
2,evet ben,evet ben de,evet ben de öyle
3,ben de,ben de öyle,evet ben de öyle
4,ne kadar,ne kadar güzel,ne kadar güzel bir
5,çok güzel,çok güzel değil,çok güzel değil mi
6,uzun zaman,çok uzun zaman,çok uzun zaman oldu
7,benim için,benim için çok,benim için çok önemli
8,uzun zaman,uzun zaman oldu,çok uzun zaman oldu
9,bu kadar,bu kadar çok,neden bu kadar çok


In [57]:
df_twogram_in_three_in_selected_four = df_twogram_in_three_in_selected_four[["fourgram","threegram","twogram"]]
df_twogram_in_three_in_selected_four

,fourgram,threegram,twogram
0,ben de seni seviyorum,ben de seni,ben de
1,evet ben de öyle,evet ben de,ben de
2,evet ben de öyle,evet ben de,evet ben
3,evet ben de öyle,ben de öyle,ben de
4,ne kadar güzel bir,ne kadar güzel,ne kadar
5,çok güzel değil mi,çok güzel değil,çok güzel
6,çok uzun zaman oldu,çok uzun zaman,uzun zaman
7,benim için çok önemli,benim için çok,benim için
8,çok uzun zaman oldu,uzun zaman oldu,uzun zaman
9,neden bu kadar çok,bu kadar çok,bu kadar


In [58]:
df_twogram_in_three_in_selected_four["fourgram"].nunique()

43

In [59]:
#df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx", index=False)  # 1B

##### Twogram In Selected Fourgram (2B)

In [60]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [61]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,benim için,benim için çok önemli
1,çok güzel,çok güzel değil mi
2,her şeyi,için her şeyi yaparım
3,çok güzel,seni görmek çok güzel
4,bu kadar,neden bu kadar uzun
...,...,...
81,biliyor musun,ne demek biliyor musun
82,evet ben,evet ben de öyle
83,senin için,senin için ne yapabilirim
84,sen mi,bunu sen mi yaptın


In [62]:
#df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_two_in_selected_four_list

In [63]:
df_two_in_selected_four2 = df_two_in_selected_four[~df_two_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_two_in_selected_four2.reset_index(drop=True, inplace=True)
df_two_in_selected_four2

,twogram,fourgram
0,her şeyi,için her şeyi yaparım
1,çok güzel,seni görmek çok güzel
2,evet mi,evet mi hayır mı
3,daha var,bir tane daha var
4,onun gibi,onun gibi bir şey
5,var mı,bir yer var mı
6,var mı,bir şeyin var mı
7,var mı,bir şeyler var mı
8,ister misin,bir şey ister misin
9,değil mi,şaka yapıyorsun değil mi


In [64]:
df_two_in_selected_four2["fourgram"].nunique()

34

In [65]:
#df_two_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx", index=False)  # 2B

##### Threegram In Selected Fourgram (4B)

In [66]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [67]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,ne kadar güzel,ne kadar güzel bir
1,ben de seni,ben de seni seviyorum
2,güzel değil mi,çok güzel değil mi
3,biliyorsun değil mi,bunu biliyorsun değil mi
4,şey var mı,bir şey var mı
5,çok uzun zaman,çok uzun zaman oldu
6,ben de öyle,evet ben de öyle
7,evet ben de,evet ben de öyle
8,bir şey var,bir şey var mı
9,bu kadar çok,neden bu kadar çok


In [68]:
#df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_three_in_selected_four_list

In [69]:
df_three_in_selected_four2 = df_three_in_selected_four[~df_three_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_three_in_selected_four2.reset_index(drop=True, inplace=True)
df_three_in_selected_four2

,threegram,fourgram


In [70]:
#df_three_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx", index=False)  # 4B

##### Twogram In Threegram (3B)

In [71]:
set_two_1B_list = list(set(df_twogram_in_three_in_selected_four["twogram"]))
set_two_2B_list = list(set(df_two_in_selected_four2["twogram"]))
set_two_list = set_two_1B_list + set_two_2B_list
#set_two_list

In [73]:
df_two_remove_used = df_twogram_used[~df_twogram_used["twogram"].isin(set_two_list)]
df_two_remove_used

,twogram
3,ve de
4,ne için
6,o kadar
9,bu çok
10,ama değil
...,...
150,öyle mi
151,para için
152,senin gibi
153,sorun değil


In [74]:
set_three_1B_list = list(set(df_twogram_in_three_in_selected_four["threegram"]))
set_three_4B_list = list(set(df_three_in_selected_four2["threegram"]))
set_three_list = set_three_1B_list+set_three_4B_list
#set_three_list

In [75]:
df_three_remove_used = df_threegram_used[~df_threegram_used["threegram"].isin(set_three_list)]
df_three_remove_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
333,bir daha hiç
334,bir de şu
335,bir daha asla
336,peki ne için


In [76]:
df_two_remove_used_list = list(set(df_two_remove_used["twogram"]))

In [77]:
# parameter
source_wordgroup_list = df_two_remove_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_remove_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [78]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three_not_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three_not_in_selected_four

,twogram,threegram
0,doğru mu,bu doğru mu
1,doğru mu,doğru mu bilmiyorum
2,doğru mu,doğru mu bu
3,doğru mu,gerçekten doğru mu
4,yeni geldi,daha yeni geldi
...,...,...
264,evet bu,evet bu harika
265,evet bu,evet bu çok
266,evet bu,evet bu da
267,evet bu,evet bu kadar


In [81]:
df_two_in_three_not_in_selected_four_freq = pd.merge(df_two_in_three_not_in_selected_four, df_threegram, how="left", on="threegram")
df_two_in_three_not_in_selected_four_freq.drop_duplicates(inplace=True)
df_two_in_three_not_in_selected_four_freq.sort_values(by="frequency", ascending=False, inplace=True)
df_two_in_three_not_in_selected_four_freq_select = df_two_in_three_not_in_selected_four_freq.head(100)
df_two_in_three_not_in_selected_four = df_two_in_three_not_in_selected_four_freq_select.drop("frequency", axis=1)
df_two_in_three_not_in_selected_four

,twogram,threegram
186,bir daha,bir daha asla
171,o zaman,tamam o zaman
79,o da,o da ne
204,bir dakika,dur bir dakika
242,daha iyi,daha iyi olur
...,...,...
37,o değil,o değil mi
101,önemli olan,önemli olan da
135,tabii ki,tabii ki öyle
158,sorun yok,hayır sorun yok


In [77]:
#df_two_in_three_not_in_selected_four_freq_select.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx", index=False)

In [83]:
df_all_result = pd.concat([df_twogram_in_three_in_selected_four,df_three_in_selected_four2,df_two_in_selected_four2,df_two_in_three_not_in_selected_four], axis=0)
df_all_result.reset_index(drop=True, inplace=True)
df_all_result

,fourgram,threegram,twogram
0,ben de seni seviyorum,ben de seni,ben de
1,evet ben de öyle,evet ben de,ben de
2,evet ben de öyle,evet ben de,evet ben
3,evet ben de öyle,ben de öyle,ben de
4,ne kadar güzel bir,ne kadar güzel,ne kadar
...,...,...,...
186,NaN,o değil mi,o değil
187,NaN,önemli olan da,önemli olan
188,NaN,tabii ki öyle,tabii ki
189,NaN,hayır sorun yok,sorun yok


In [84]:
df_all_result["fourgram"].nunique()

77

##### Section 5

In [78]:
set_two_all = set_two_list + list(set(df_two_in_three_not_in_selected_four["twogram"]))
#set_two_all

In [79]:
df_twogram_alone = df_twogram_remove_used[~df_twogram_remove_used["twogram"].isin(set_two_all)]
df_twogram_alone.reset_index(drop=True, inplace=True)
df_twogram_alone

,twogram,frequency
0,nasılsın dostum,2516
1,merhaba nasılsın,2149
2,hayır değildi,2027
3,neden acaba,1617
4,selam kızlar,1476
...,...,...
84816,dışarıda bizimle,6
84817,bizim kimseye,6
84818,tarafa bunu,6
84819,beri kahrolası,6


In [80]:
df_twogram_alone_select = df_twogram_alone.head(100)
df_twogram_alone_select

,twogram,frequency
0,nasılsın dostum,2516
1,merhaba nasılsın,2149
2,hayır değildi,2027
3,neden acaba,1617
4,selam kızlar,1476
...,...,...
95,hayır yoktu,547
96,hey merhaba,545
97,evet anlaşıldı,542
98,durun bekleyin,540


In [81]:
df_twogram_alone_select.to_excel(f"{lang_folder.capitalize()}_Twogram_Alone_(5)_Result.xlsx", index=False)  # 5

##### Section 6

In [82]:
set_three_all = set_three_list + list(set(df_two_in_three_not_in_selected_four["threegram"]))

In [83]:
df_threegram_alone = df_threegram_remove_used[~df_threegram_remove_used["threegram"].isin(set_three_all)]
df_threegram_alone.reset_index(drop=True, inplace=True)
df_threegram_alone

,threegram,frequency
0,daha iyi bir,35136
1,o kadar da,33638
2,çok güzel bir,30839
3,çok iyi bir,29751
4,ne kadar da,18799
...,...,...
1689,o zaman dön,10
1690,seninle tanrının arasında,10
1691,şey yolunda dedim,10
1692,gitsin öyle mi,10


In [84]:
df_threegram_alone_select = df_threegram_alone.head(100)
df_threegram_alone_select

,threegram,frequency
0,daha iyi bir,35136
1,o kadar da,33638
2,çok güzel bir,30839
3,çok iyi bir,29751
4,ne kadar da,18799
...,...,...
95,başka bir şeyin,1084
96,kötü bir şeyler,1057
97,için çok uzun,1056
98,bu ne güzel,1031


In [85]:
df_threegram_alone_select.to_excel(f"{lang_folder.capitalize()}_Threegram_Alone_(6)_Result.xlsx", index=False)  # 6

##### Section 7

In [86]:
set_four_all = list(set(df_three_in_selected_four2["fourgram"])) + list(set(df_two_in_selected_four2["fourgram"])) + list(set(df_twogram_in_three_in_selected_four["fourgram"]))

In [87]:
df_fourgram_alone = df_fourgram_selected_for_course[~df_fourgram_selected_for_course["fourgram"].isin(set_four_all)]
df_fourgram_alone.reset_index(drop=True, inplace=True)
df_fourgram_alone

,fourgram,frequency


In [88]:
df_fourgram_alone_select = df_fourgram_alone.head(100)
df_fourgram_alone_select

,fourgram,frequency


In [89]:
df_fourgram_alone_select.to_excel(f"{lang_folder.capitalize()}_Fourgram_Alone_(7)_Result.xlsx", index=False) 

#### Copy Move And Delete

In [90]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Twogram_Alone_(5)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_(1A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Selected_Fourgram_(2A)_Result.xlsx',
 'Turkish_Threegram_Used_In_Selected_Fourgram_(4A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_Not_In_Selected_Fourgram_(3A)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx',
 'Turkish_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx',
 'Turkish_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx',
 'Turkish_Threegram_Alone_(6)_Result.xlsx',
 'Turkish_Fourgram_Alone_(7)_Result.xlsx']

In [91]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [92]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass